In [ ]:
from bs4 import BeautifulSoup
import requests
from requests import session, get
from urllib.parse import urljoin
import shutil

from typing import Dict, List

**1. Verschaffe dir mittels Chromes Entwicklertools einen Überblick über die Loginform auf [dieser](https://www.wim.uni-mannheim.de/schlather/teaching/aktuelle-semester/) Seite. Welche Daten werden alles benötigt und abgesendet?**

**2. Schreibe eine Funktion `materialien_login() -> requests.session`, welche sich auf der Seite einloggt und schließlich ein `request.session`-Objekt zurückgibt, welches u.A. alle Cookies enthält.** Achtung: es gibt zwei HTML-Forms auf der Seite.

In [ ]:
def materialien_login() -> requests.session:
    login_url = "https://www.wim.uni-mannheim.de/schlather/teaching/aktuelle-semester/"
    soup = BeautifulSoup(get(login_url).text, "lxml")
    # Erstelle session objekt für Login
    sess = requests.session()
    # Logindaten
    form_data = {
        'user': '...',  # <--- Kommilitonen fragen :)
        'pass': '...',  # <---       -"-
        'submit': 'Anmelden',
        'logintype': 'login',
        'pid': 0,
        'redirect_url': '/schlather/teaching/aktuelle-semester/materialien/',
        'tx_felogin_pi1[noredirect]': 0
    }
    # Login
    sess.post(login_url, data=form_data)
    # Gibt das Sessionobjekt zurück
    return sess

**3. Schreibe eine Funktion `scrape_pdfs(sess: requests.session) -> Dict[str, List[str]]`, welche für jede Vorlesung auf der Materialenseite die Links zu allen pdf-Dateien findet. Die Funktion erwartet als einziges Argument ein `requests.session` Objekt, das bereits auf der Seite eingeloggt ist. Die Funktion soll ein dictionary mit allen .pdf-Links pro Vorlesung zurückgeben.**

Hinweis: Verwende `.replace("\xad", "")` um *HTML entities* aus dem Vorlesungsnamen (`str`) zu entfernen.

In [ ]:
def scrape_pdfs(sess: requests.session) -> Dict[str, List[str]]:
    base_url = "https://www.wim.uni-mannheim.de/schlather/teaching/aktuelle-semester/materialien/"
    soup = BeautifulSoup(sess.get(base_url).text, "lxml")
    # Jede Vorlesung ist in einer 'gridelement accordion-item anchor' CSS Klasse:
    res = {}
    for vl in soup.find_all(class_="gridelement accordion-item anchor"):
        vl_name = vl.a.h4.text.replace("\xad", "")
        # Finde alle .pdf Dateien: (alternativ: href=re.compile(r"pdf$"))
        pdf_files = [ ]
        for link in vl.find_all("a", href=lambda h: h is not None and h.endswith("pdf")):
            pdf_files.append(urljoin(base_url, link["href"]))
        # Füge die Liste der .pdf-Links in das dict ein:
        res[vl_name] = pdf_files
    # Fertig
    sess.close()
    return res

**4. Teste deinen Code, ob auch die Links richtig *geparsed* werden:**

In [ ]:
scrape_pdfs(materialien_login())

**5. Verwende die nachfolgende Funktion, um alle Dateien herunterzuladen:**

In [ ]:
import pathlib

def download_file(sess: requests.session, url:str, pfad: pathlib.Path, filename: str) -> None:
    req = get(url, stream=True)
    if req.status_code == 200:
        with open(pfad / filename, "wb") as f:
            shutil.copyfileobj(req.raw, f)
                
def main(verbose=False) -> None:
    # Scrape die Daten
    sess = materialien_login()
    daten = scrape_pdfs(sess)

    # Lade jetzt die Dateien je Vorlesung
    for vl_name, links in daten.items():
        # Erstelle Ordner für VL, falls dieser noch nicht existiert
        p = pathlib.Path().cwd() / vl_name.replace(":", "")
        if not p.exists():
            p.mkdir()
        # Lade Dateien runter
        for link in links:
            dateiname = link.split("/")[-1]
            if verbose:
                print(f"Lade [{vl_name}]: {dateiname}...")
            download_file(sess, link, p, dateiname)

In [ ]:
main(verbose=True)

**Zusatz: Das `sess` Objekt an jede Funktion mitzuschleppen ist irgendwie nervig und nicht unbedingt *pythonic*. Für größere Projekte ist es daher ratsam, sich eine Klasse zu schreiben**